In [0]:
import sys
sys.path.append("../../")

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from src_staging.sv_policy_stage import create_policy_stage, sat_policy_status_event_schema

In [0]:
partition_name = 'partition_date'
partition_date = '20250828'

In [0]:
_bz_table = 'ctl_central_published.sc_bz_source_name.dt_customer_journey_daily'
_sat_sv_table = 'ctl_central_published.sc_sv_raw_vault.sat_policy_status_event'
partition_date_fmt = f"{partition_date[:4]}-{partition_date[4:6]}-{partition_date[6:]}"

In [0]:
sv_df = create_policy_stage(_bz_table, partition_name, partition_date_fmt)
sv_df.display()

In [0]:
status_events = (sv_df
  .withColumn("evt", explode_outer(col("status_history")))
  .select(
    col("policy_number"),
    upper(trim(col("evt.status"))).alias("status"),
    to_timestamp(col("evt.ts")).alias("status_ts"),
    "record_source",
    "partition_date"
  )
)
status_events.createOrReplaceTempView("incoming_policy_status_event")

In [0]:
if not spark.catalog.tableExists(_sat_sv_table):
    (spark.createDataFrame([], sat_policy_status_event_schema())
         .write.format("delta")
         .saveAsTable(_sat_sv_table))

In [0]:
spark.sql(f"""
        MERGE INTO {_sat_sv_table} AS tgt
        USING incoming_policy_status_event AS src
        ON  tgt.policy_number = src.policy_number
        AND tgt.status_ts     = src.status_ts
        AND tgt.status        = src.status
        WHEN NOT MATCHED THEN
        INSERT (policy_number, status, status_ts, record_source, _ingest_ts, partition_date)
        VALUES (src.policy_number, src.status, src.status_ts, src.record_source, current_timestamp(), src.partition_date)"""
)